In [ ]:
The front office has brought in catcher f06c9fdf this offseason. 
Using the data provided, please prepare a report for a coaching staff audience to 
brief them on the player’s current strengths and weaknesses as a pitch receiver. 
The report (explanations and any visuals) should be less than two pages long. 
Please also submit the code used to generate the brief, though your process here is 
less important than the way you communicate your finding in the report.


In [1]:
import csv 
import pandas as pd 
import numpy as np
import collections

import matplotlib.pyplot as plt
%matplotlib inline

fig=plt.figure(figsize=(14, 12), dpi= 80, facecolor='w', edgecolor='k')

pd.set_option('display.max_columns', None)

<Figure size 1120x960 with 0 Axes>

In [174]:
df = pd.read_csv('2020-train.csv')
df.head()

,pitcher_id,pitcher_side,batter_id,batter_side,stadium_id,umpire_id,catcher_id,inning,top_bottom,outs,balls,strikes,release_speed,vert_release_angle,horz_release_angle,spin_rate,spin_axis,tilt,rel_height,rel_side,extension,vert_break,induced_vert_break,horz_break,plate_height,plate_side,zone_speed,vert_approach_angle,horz_approach_angle,zone_time,x55,y55,z55,pitch_type,pitch_call,pitch_id
0,d7e3acce,Right,32678d8d,Right,a4833794,f88d09f4,83cdf9ff,3,1,0.0,0,1,93.008102,-1.990750,-0.364950,1995.780029,-150.653000,1:00,5.97789,0.221744,6.45304,-12.477000,18.268900,9.38941,3.05034,0.555689,86.024200,-4.37258,1.429580,0.404622,-0.059343,55,6.03322,FA,InPlay,42fce2f6
1,44ec1bf5,Right,81d51733,Left,f60d6ea5,b67d862c,a126f66f,6,2,0.0,3,2,97.935898,-2.916730,-2.695220,2388.689941,-170.166000,12:15,6.18328,2.123090,6.37298,-10.453700,18.147200,2.89933,2.61284,-0.111897,89.458199,-4.90467,-2.148410,0.385719,-2.148680,55,6.23380,FA,InPlay,3e9cda86
2,44d87ee6,Left,8eefccb7,Right,a9b8b538,13993d26,9db4e46f,5,2,2.0,3,1,81.831398,-1.576020,1.045500,1966.949951,159.830994,11:15,6.12413,-1.294740,6.13114,-23.592199,17.337099,-6.18609,2.63461,-0.937995,75.593597,-6.00728,-0.122044,0.463953,1.300450,55,6.14750,CH,BallCalled,f129a6cd
3,ff6adae0,Right,8f8ab5af,Right,e569ec39,0d8ba4bb,bbbfd290,5,1,2.0,1,2,83.775703,-0.387083,-0.613163,2539.459961,54.268398,7:45,6.58357,1.665380,5.59444,-49.169498,-8.342350,-10.58620,2.17409,0.233674,76.396400,-9.50640,-2.581980,0.458471,-1.659590,55,6.60043,CU,InPlay,03e9bc05
4,c70c96e5,Right,10874746,Right,a5ce1bf6,94a4c552,75087ec8,8,1,2.0,2,1,91.208000,0.140618,-4.035290,2409.110107,246.348999,2:15,4.77355,1.495840,6.12665,-24.760099,8.488800,19.89060,2.84640,-0.703636,83.215302,-4.53233,-0.268188,0.415965,-1.526170,55,4.77332,FA,StrikeCalled,48feb675


In [175]:
df['ball_strike'] = np.where(df['pitch_call'].isin (['HitByPitch','BallCalled','BallIntentional']), 0 , 1)

In [297]:
dfcatch = df[df['catcher_id'] =='f06c9fdf']
dfcatcho = df[df['catcher_id'] !='f06c9fdf']

In [180]:
print(dfcatch.shape,dfcatcho.shape)

(8822, 37) (573383, 37)


In [298]:
#Only concerned about BallCalled & StrikeCalled
x = dfcatch[(dfcatch['pitch_call'] == 'StrikeCalled') | (dfcatch['pitch_call'] == 'BallCalled')]
xo = dfcatcho[(dfcatcho['pitch_call'] == 'StrikeCalled') | (dfcatcho['pitch_call'] == 'BallCalled')]

In [83]:
x.shape, xo.shape

((4832, 37), (302984, 37))

## Called Strikes by Zone: All, Pitch Type, Pitcher Hand, Batter Hand

Strike Zone from Catcher's Perspective
<img src='zone_cells.png'>

Strike zone dimensions         
Plate width: 17 inches = 1.41666 feet    
+/- 0.71 ft from center of plate        
Average height of bottom of zone: 1.5 ft
Average height of top of zone: 3.5ft    
2.5ft +/- 1 ft


In [294]:
def all(temp1, temp1o):
    all_ball = temp1.groupby(['pitch_call']).agg({'pitch_call':'count'})
    all_type = temp1.agg({'pitch_call': 'count'})
    temp = all_ball.div(all_type,level = 'pitch_call')
    print ('Catcher: {}'.format (collections.Counter(temp1['pitch_call']).most_common()))
    
    all_ball = temp1o.groupby(['pitch_call']).agg({'pitch_call':'count'})
    all_type = temp1o.agg({'pitch_call': 'count'})
    tempo = all_ball.div(all_type,level = 'pitch_call')
    print ('Others: {}'.format (collections.Counter(temp1o['pitch_call']).most_common()))
    
    diff = (temp-tempo) * 100
    total = pd.concat([temp, tempo, diff], axis=1, sort=False).dropna()
    print (total)
    
def type(temp1, temp1o):
    pitch_type_ball = temp1.groupby(['pitch_type','pitch_call']).agg({'pitch_call':'count'})
    pitch_type = temp1.groupby(['pitch_type']).agg({'pitch_call': 'count'})
    temp = pitch_type_ball.div(pitch_type,level = 'pitch_type')
    print ('Catcher: {}'.format (collections.Counter(temp1['pitch_type']).most_common()))
    
    pitch_type_ball = temp1o.groupby(['pitch_type','pitch_call']).agg({'pitch_call':'count'})
    pitch_type= temp1o.groupby(['pitch_type']).agg({'pitch_call': 'count'})
    tempo = pitch_type_ball.div(pitch_type,level = 'pitch_type')
    print ('Others: {}'.format (collections.Counter(temp1o['pitch_type']).most_common()))
    
    diff = (temp - tempo) * 100
    total = pd.concat([temp, tempo, diff], axis=1, sort=False).dropna()
    print (total)
    
def pitcher(temp1, temp1o):
    pitcher_side_ball = temp1.groupby(['pitcher_side','pitch_call']).agg({'pitch_call':'count'})
    pitcher_side = temp1.groupby(['pitcher_side']).agg({'pitch_call': 'count'})
    temp = pitcher_side_ball.div(pitcher_side,level = 'pitcher_side')
    print ('Catcher: {}'.format (collections.Counter(temp1['pitcher_side']).most_common()))
    
    pitcher_side_ball = temp1o.groupby(['pitcher_side','pitch_call']).agg({'pitch_call':'count'})
    pitcher_side= temp1o.groupby(['pitcher_side']).agg({'pitch_call': 'count'})
    tempo = pitcher_side_ball.div(pitcher_side,level = 'pitcher_side')
    print ('Others: {}'.format (collections.Counter(temp1o['pitcher_side']).most_common()))
    
    diff = (temp - tempo) * 100
    total = pd.concat([temp, tempo, diff], axis=1, sort=False).dropna()
    print (total)
         
def batter(temp1, temp1o):
    batter_side_ball = temp1.groupby(['batter_side','pitch_call']).agg({'pitch_call':'count'})
    batter_side = temp1.groupby(['batter_side']).agg({'pitch_call': 'count'})
    temp = batter_side_ball.div(batter_side,level = 'batter_side')
    print ('Catcher: {}'.format (collections.Counter(temp1['batter_side']).most_common()))
        
    batter_side_ball = temp1o.groupby(['batter_side','pitch_call']).agg({'pitch_call':'count'})
    batter_side = temp1o.groupby(['batter_side']).agg({'pitch_call': 'count'})
    tempo = batter_side_ball.div(batter_side,level = 'batter_side')
    print ('Others: {}'.format (collections.Counter(temp1o['batter_side']).most_common()))
        
    diff = (temp - tempo) * 100
    total = pd.concat([temp, tempo, diff], axis=1, sort=False).dropna()
    print (total)

### Zone 11

In [299]:
temp3 = x[(x['plate_height'] >2.5) & (x['plate_side'] >.71) ]
temp2 = x[(x['plate_height'] >3.5) & (x['plate_side'] <.71)&(x['plate_side'] > 0) ]
temp3o = xo[(xo['plate_height'] > 2.5)  & (xo['plate_side'] >.71)]
temp2o = xo[(xo['plate_height'] >3.5) & (xo['plate_side'] <.71)&(xo['plate_side'] > 0) ]
temp1 = pd.concat([temp3,temp2])
temp1o = pd.concat([temp3o,temp2o])

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('BallCalled', 704), ('StrikeCalled', 79)]
Others: [('BallCalled', 42303), ('StrikeCalled', 4777)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.899106    0.898534    0.057159
StrikeCalled    0.100894    0.101466   -0.057159
Catcher: [('FA', 516), ('CH', 111), ('SL', 80), ('CU', 74), ('XX', 2)]
Others: [('FA', 32289), ('SL', 6143), ('CU', 4327), ('CH', 4171), ('KN', 81), ('XX', 36), (nan, 33)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.882883    0.905538   -2.265536
           StrikeCalled    0.117117    0.094462    2.265536
CU         BallCalled      0.864865    0.861336    0.352907
           StrikeCalled    0.135135    0.138664   -0.352907
FA         BallCalled      0.910853    0.908514    0.233898
           StrikeCalled    0.089147    0.091486   -0.233898
SL         BallCalled      0.875000    0.86

### Zone 12

In [300]:
temp3 = x[(x['plate_height'] >2.5) & (x['plate_side'] <-.71) ]
temp2 = x[(x['plate_height'] >3.5) & (x['plate_side'] >-.71)&(x['plate_side'] < 0) ]
temp3o = xo[(dfcatcho['plate_height'] > 2.5)  & (xo['plate_side'] <-.71)]
temp2o = xo[(xo['plate_height'] >3.5) & (xo['plate_side'] >-.71)&(xo['plate_side'] < 0) ]
temp1 = pd.concat([temp3,temp2])
temp1o = pd.concat([temp3o,temp2o])

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Catcher: [('BallCalled', 436), ('StrikeCalled', 66)]
Others: [('BallCalled', 31698), ('StrikeCalled', 4654)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.868526    0.871974   -0.344814
StrikeCalled    0.131474    0.128026    0.344814
Catcher: [('FA', 317), ('SL', 94), ('CH', 70), ('CU', 20), ('XX', 1)]
Others: [('FA', 25435), ('SL', 5763), ('CU', 2595), ('CH', 2408), ('KN', 89), ('XX', 34), (nan, 28)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.885714    0.873754    1.196013
           StrikeCalled    0.114286    0.126246   -1.196013
CU         BallCalled      0.900000    0.823507    7.649326
           StrikeCalled    0.100000    0.176493   -7.649326
FA         BallCalled      0.886435    0.885316    0.111982
           StrikeCalled    0.113565    0.114684   -0.111982
SL         BallCalled      0.787234    0.835

### Zone 13

In [301]:
temp3 = x[(x['plate_height'] <2.5) & (x['plate_side'] >.71) ]
temp2 = x[(x['plate_height'] <1.5) & (x['plate_side'] <.71)&(x['plate_side'] > 0) ]
temp3o = xo[(xo['plate_height'] <2.5)  & (xo['plate_side'] >.71)]
temp2o = xo[(xo['plate_height'] <1.5) & (xo['plate_side'] <.71)&(xo['plate_side'] > 0) ]
temp1 = pd.concat([temp3,temp2])
temp1o = pd.concat([temp3o,temp2o])

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('BallCalled', 1001), ('StrikeCalled', 114)]
Others: [('BallCalled', 50262), ('StrikeCalled', 7911)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.897758    0.864009    3.374877
StrikeCalled    0.102242    0.135991   -3.374877
Catcher: [('FA', 510), ('SL', 253), ('CH', 228), ('CU', 124)]
Others: [('FA', 27932), ('SL', 12650), ('CH', 10438), ('CU', 7079), (nan, 47), ('XX', 20), ('KN', 7)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.956140    0.931692    2.444846
           StrikeCalled    0.043860    0.068308   -2.444846
CU         BallCalled      0.943548    0.888402    5.514607
           StrikeCalled    0.056452    0.111598   -5.514607
FA         BallCalled      0.837255    0.821388    1.586725
           StrikeCalled    0.162745    0.178612   -1.586725
SL         BallCalled      0.944664    0.888538  

### Zone 14

In [302]:
temp3 = x[(x['plate_height'] <2.5) & (x['plate_side'] <-.71) ]
temp2 = x[(x['plate_height'] <1.5) & (x['plate_side'] >-.71)&(x['plate_side'] < 0) ]
temp3o = xo[(dfcatcho['plate_height'] < 2.5)  & (xo['plate_side'] <-.71)]
temp2o = xo[(xo['plate_height'] <1.5) & (xo['plate_side'] >-.71)&(xo['plate_side'] < 0) ]
temp1 = pd.concat([temp3,temp2])
temp1o = pd.concat([temp3o,temp2o])

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


Catcher: [('BallCalled', 1154), ('StrikeCalled', 79)]
Others: [('BallCalled', 74164), ('StrikeCalled', 8987)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.935929     0.89192    4.400911
StrikeCalled    0.064071     0.10808   -4.400911
Catcher: [('FA', 480), ('SL', 371), ('CH', 192), ('CU', 190)]
Others: [('FA', 34749), ('SL', 25968), ('CU', 11569), ('CH', 10730), ('KN', 64), (nan, 52), ('XX', 19)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.979167    0.942404    3.676219
           StrikeCalled    0.020833    0.057596   -3.676219
CU         BallCalled      0.973684    0.940704    3.298061
           StrikeCalled    0.026316    0.059296   -3.298061
FA         BallCalled      0.904167    0.836110    6.805628
           StrikeCalled    0.095833    0.163890   -6.805628
SL         BallCalled      0.935310    0.923868 

### Zone 1 

In [307]:
temp1 = x[(x['plate_height'] >2.833) & (x['plate_height'] < 3.5) & \
          (x['plate_side'] <.71) & (x['plate_side'] > 0.47) ]
temp1o = xo[(xo['plate_height'] >2.833) & (xo['plate_height'] < 3.5) & \
          (xo['plate_side'] <.71) & (xo['plate_side'] > 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 57), ('BallCalled', 14)]
Others: [('StrikeCalled', 2788), ('BallCalled', 909)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.197183    0.245875   -4.869194
StrikeCalled    0.802817    0.754125    4.869194
Catcher: [('FA', 45), ('SL', 13), ('CU', 8), ('CH', 5)]
Others: [('FA', 2178), ('SL', 741), ('CU', 528), ('CH', 243), ('KN', 3), ('XX', 2), (nan, 2)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.200000    0.189300    1.069959
           StrikeCalled    0.800000    0.810700   -1.069959
CU         BallCalled      0.125000    0.210227   -8.522727
           StrikeCalled    0.875000    0.789773    8.522727
FA         BallCalled      0.244444    0.267677   -2.323232
           StrikeCalled    0.755556    0.732323    2.323232
SL         BallCalled      0.076923    0.225371  -14.844804
        

### Zone 2 

In [308]:
temp1 = x[(x['plate_height'] >2.833) & (x['plate_height'] < 3.5) & \
          (x['plate_side'] >-.47) & (x['plate_side'] < 0.47) ]
temp1o = xo[(xo['plate_height'] >2.833) & (xo['plate_height'] < 3.5) & \
          (xo['plate_side'] >-.47) & (xo['plate_side'] < 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 182), ('BallCalled', 24)]
Others: [('StrikeCalled', 10503), ('BallCalled', 1791)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.116505    0.145681   -2.917597
StrikeCalled    0.883495    0.854319    2.917597
Catcher: [('FA', 126), ('SL', 47), ('CU', 24), ('CH', 9)]
Others: [('FA', 7069), ('SL', 2800), ('CU', 1752), ('CH', 638), ('KN', 26), (nan, 7), ('XX', 2)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         StrikeCalled    1.000000    0.899687   10.031348
CU         BallCalled      0.083333    0.122146   -3.881279
           StrikeCalled    0.916667    0.877854    3.881279
FA         BallCalled      0.142857    0.164521   -2.166401
           StrikeCalled    0.857143    0.835479    2.166401
SL         BallCalled      0.085106    0.123929   -3.882219
           StrikeCalled    0.914894    0.876071    3.882219


### Zone 3 

In [309]:
temp1 = x[(x['plate_height'] >2.833) & (x['plate_height'] < 3.5) & \
          (x['plate_side'] < -.47) & (x['plate_side'] > -0.71) ]
temp1o = xo[(xo['plate_height'] >2.833) & (xo['plate_height'] < 3.5) & \
          (xo['plate_side'] < -.47) & (xo['plate_side'] > -0.71) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 37), ('BallCalled', 16)]
Others: [('StrikeCalled', 2568), ('BallCalled', 827)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.301887    0.243594    5.829327
StrikeCalled    0.698113    0.756406   -5.829327
Catcher: [('FA', 32), ('SL', 11), ('CU', 5), ('CH', 5)]
Others: [('FA', 2169), ('SL', 705), ('CU', 359), ('CH', 149), ('KN', 8), (nan, 4), ('XX', 1)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.200000    0.167785    3.221477
           StrikeCalled    0.800000    0.832215   -3.221477
CU         BallCalled      0.200000    0.178273    2.172702
           StrikeCalled    0.800000    0.821727   -2.172702
FA         BallCalled      0.343750    0.272476    7.127420
           StrikeCalled    0.656250    0.727524   -7.127420
SL         BallCalled      0.272727    0.207092    6.563507
        

### Zone 4

In [310]:
temp1 = x[(x['plate_height'] <2.833) & (x['plate_height'] > 2.166) & \
          (x['plate_side'] <.71) & (x['plate_side'] > 0.47) ]
temp1o = xo[(xo['plate_height'] <2.833) & (xo['plate_height'] > 2.166) & \
          (xo['plate_side'] <.71) & (xo['plate_side'] > 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 91), ('BallCalled', 3)]
Others: [('StrikeCalled', 4464), ('BallCalled', 91)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.031915    0.019978    1.193685
StrikeCalled    0.968085    0.980022   -1.193685
Catcher: [('FA', 54), ('CU', 15), ('SL', 14), ('CH', 11)]
Others: [('FA', 2702), ('SL', 944), ('CU', 636), ('CH', 266), (nan, 4), ('KN', 2), ('XX', 1)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         StrikeCalled    1.000000    0.977444    2.255639
CU         BallCalled      0.066667    0.014151    5.251572
           StrikeCalled    0.933333    0.985849   -5.251572
FA         BallCalled      0.037037    0.022206    1.483126
           StrikeCalled    0.962963    0.977794   -1.483126
SL         StrikeCalled    1.000000    0.985169    1.483051
Catcher: [('Right', 60), ('Left', 34)]
Others: [('Right', 3318), ('L

### ZONE 5

In [311]:
temp1 = x[(x['plate_height'] <2.833) & (x['plate_height'] > 2.166) & \
          (x['plate_side'] >-.47) & (x['plate_side'] < 0.47) ]
temp1o = xo[(xo['plate_height'] <2.833) & (xo['plate_height'] > 2.166) & \
          (xo['plate_side'] >-.47) & (xo['plate_side'] < 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)                                                                                                     

Catcher: [('StrikeCalled', 255)]
Others: [('StrikeCalled', 17240), ('BallCalled', 24)]
              pitch_call  pitch_call  pitch_call
StrikeCalled         1.0     0.99861    0.139018
Catcher: [('FA', 158), ('SL', 49), ('CU', 27), ('CH', 21)]
Others: [('FA', 9960), ('SL', 3765), ('CU', 2499), ('CH', 1005), (nan, 17), ('KN', 14), ('XX', 4)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         StrikeCalled         1.0    0.997015    0.298507
CU         StrikeCalled         1.0    0.999200    0.080032
FA         StrikeCalled         1.0    0.998695    0.130522
SL         StrikeCalled         1.0    0.998938    0.106242
Catcher: [('Right', 160), ('Left', 95)]
Others: [('Right', 12430), ('Left', 4833), ('R', 1)]
                           pitch_call  pitch_call  pitch_call
pitcher_side pitch_call                                      
Left         StrikeCalled         1.0    0.999793    0.020691
Right        Strik

### Zone 6

In [312]:
temp1 = x[(x['plate_height'] <2.833) & (x['plate_height'] > 2.166) & \
          (x['plate_side'] < -.47) & (x['plate_side'] > -0.71) ]
temp1o = xo[(xo['plate_height'] <2.833) & (xo['plate_height'] > 2.166) & \
          (xo['plate_side'] < -.47) & (xo['plate_side'] > -0.71) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)                                                                                                     

Catcher: [('StrikeCalled', 68), ('BallCalled', 6)]
Others: [('StrikeCalled', 4861), ('BallCalled', 137)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.081081    0.027411    5.367012
StrikeCalled    0.918919    0.972589   -5.367012
Catcher: [('FA', 49), ('SL', 17), ('CU', 6), ('CH', 2)]
Others: [('FA', 3229), ('SL', 997), ('CU', 523), ('CH', 237), ('KN', 7), (nan, 4), ('XX', 1)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         StrikeCalled    1.000000    0.978903    2.109705
CU         StrikeCalled    1.000000    0.982792    1.720841
FA         BallCalled      0.102041    0.032208    6.983270
           StrikeCalled    0.897959    0.967792   -6.983270
SL         BallCalled      0.058824    0.019057    3.976636
           StrikeCalled    0.941176    0.980943   -3.976636
Catcher: [('Right', 52), ('Left', 22)]
Others: [('Right', 3543), ('Le

### ZONE 7

In [313]:
temp1 = x[(x['plate_height'] >1.5) & (x['plate_height'] < 2.166) & \
          (x['plate_side'] <.71) & (x['plate_side'] > 0.47) ]
temp1o = xo[(xo['plate_height'] >1.5) & (xo['plate_height'] < 2.166) & \
          (xo['plate_side'] <.71) & (xo['plate_side'] > 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 64), ('BallCalled', 18)]
Others: [('StrikeCalled', 4232), ('BallCalled', 713)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.219512    0.144186    7.532615
StrikeCalled    0.780488    0.855814   -7.532615
Catcher: [('FA', 51), ('SL', 18), ('CH', 8), ('CU', 5)]
Others: [('FA', 3026), ('SL', 924), ('CU', 542), ('CH', 449), (nan, 2), ('KN', 2)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.250000    0.202673    4.732739
           StrikeCalled    0.750000    0.797327   -4.732739
CU         BallCalled      0.200000    0.158672    4.132841
           StrikeCalled    0.800000    0.841328   -4.132841
FA         BallCalled      0.176471    0.128222    4.824851
           StrikeCalled    0.823529    0.871778   -4.824851
SL         BallCalled      0.333333    0.159091   17.424242
           StrikeCa

### Zone 8

In [314]:
temp1 = x[(x['plate_height'] <2.166) & (x['plate_height'] > 1.5) & \
          (x['plate_side'] >-.47) & (x['plate_side'] < 0.47) ]
temp1o = xo[(xo['plate_height'] <2.166) & (xo['plate_height'] > 1.5) & \
          (xo['plate_side'] >-.47) & (xo['plate_side'] < 0.47) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)

Catcher: [('StrikeCalled', 234), ('BallCalled', 53)]
Others: [('StrikeCalled', 18683), ('BallCalled', 2218)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.184669    0.106119    7.854967
StrikeCalled    0.815331    0.893881   -7.854967
Catcher: [('FA', 190), ('SL', 43), ('CH', 28), ('CU', 26)]
Others: [('FA', 12814), ('SL', 3983), ('CU', 2396), ('CH', 1682), (nan, 13), ('KN', 9), ('XX', 4)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.428571    0.140904   28.766774
           StrikeCalled    0.571429    0.859096  -28.766774
CU         BallCalled      0.115385    0.118531   -0.314627
           StrikeCalled    0.884615    0.881469    0.314627
FA         BallCalled      0.163158    0.099579    6.357931
           StrikeCalled    0.836842    0.900421   -6.357931
SL         BallCalled      0.162791    0.104946    5.7844

### Zone 9

In [315]:
temp1 = x[(x['plate_height'] >1.5) & (x['plate_height'] < 2.166) & \
          (x['plate_side'] < -.47) & (x['plate_side'] > -0.71) ]
temp1o = xo[(xo['plate_height'] >1.5) & (xo['plate_height'] < 2.166) & \
          (xo['plate_side'] < -.47) & (xo['plate_side'] > -0.71) ]

all(temp1,temp1o)
type(temp1,temp1o)
pitcher(temp1,temp1o)
batter(temp1,temp1o)  

Catcher: [('StrikeCalled', 55), ('BallCalled', 22)]
Others: [('StrikeCalled', 5043), ('BallCalled', 1088)]
              pitch_call  pitch_call  pitch_call
pitch_call                                      
BallCalled      0.285714    0.177459   10.825547
StrikeCalled    0.714286    0.822541  -10.825547
Catcher: [('FA', 48), ('SL', 19), ('CU', 6), ('CH', 4)]
Others: [('FA', 3892), ('SL', 1247), ('CU', 567), ('CH', 415), ('KN', 7), (nan, 2), ('XX', 1)]
                         pitch_call  pitch_call  pitch_call
pitch_type pitch_call                                      
CH         BallCalled      0.500000    0.289157   21.084337
           StrikeCalled    0.500000    0.710843  -21.084337
CU         BallCalled      0.166667    0.195767   -2.910053
           StrikeCalled    0.833333    0.804233    2.910053
FA         BallCalled      0.270833    0.156989   11.384464
           StrikeCalled    0.729167    0.843011  -11.384464
SL         BallCalled      0.315789    0.195670   12.011987
      